# Apêndice 

Códigos auxiliares para geração de gráficos e informaçõea adicionais.

#### Mosaicos Jogador vs. MCTS

In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from game import ConnectState
from connected_four import ConnectState, GameMeta, MCTSMeta
from mcts import MCTS


def render_connect4_board(board_array, ax):
    """
    Desenha o tabuleiro de Connect4 com origem (0,0) no canto inferior-esquerdo.
    Ajusta para que row 0 seja o topo do board_array.
    """
    ax.set_facecolor('#1f77b4')
    for r in range(6):
        for c in range(7):
            # Inverte eixo y: r=0 (bottom) mapeia board_array[5]
            cell = board_array[5 - r][c]
            color = 'white'
            if cell == 1:
                color = 'red'
            elif cell == 2:
                color = 'yellow'
            center = (c, r)
            circle = Circle(center, 0.4, edgecolor='black', facecolor=color)
            ax.add_patch(circle)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(-0.5, 6.5)
    ax.set_ylim(-0.5, 5.5)


def save_mosaic_from_states(states, picks, output_path='mosaic.png'):
    """
    Gera um mosaico de imagens a partir de uma lista de estados de jogo.
    """
    rows = cols = math.ceil(math.sqrt(len(picks)))
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 3, rows * 2.5))
    axs = axs.flatten()

    for idx, move_idx in enumerate(picks):
        board = states[move_idx - 1]
        ax = axs[idx]
        render_connect4_board(board, ax)
        ax.set_title(f'Jogada {move_idx}', pad=5)

    for ax in axs[len(picks):]:
        ax.axis('off')

    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path) or '.', exist_ok=True)
    plt.savefig(output_path)
    plt.close(fig)


def human_vs_mcts(picks=None, time_limit=1.0, output_path='mosaic.png'):
    """
    Joga uma partida humano vs MCTS interativo e salva mosaico a cada 3 jogadas + final.

    Args:
        picks (List[int], opcional): sobrescreve capturas automáticas.
        time_limit (float): segundos de MCTS por jogada do computador.
        output_path (str): arquivo de saída do mosaico.
    """
    state = ConnectState()
    states = []   # snapshots de board arrays
    current_player = 1
    move_count = 0

    while not state.game_over():
        move_count += 1
        state.print()

        if current_player == 1:
            # Jogada humana
            valid = False
            while not valid:
                try:
                    col = int(input("Sua vez (vermelho). Escolha coluna 0-6: "))
                    if col in state.get_legal_moves():
                        valid = True
                    else:
                        print("Coluna inválida ou cheia. Tente outra.")
                except ValueError:
                    print("Entrada inválida. Insira um número entre 0 e 6.")
            move = col
        else:
            # Jogada MCTS
            print("MCTS a pensar...")
            mcts = MCTS(state)
            mcts.search(time_limit)
            move = mcts.best_move()
            print(f"MCTS joga na coluna {move}.")

        # Aplica movimento e salva snapshot
        state.move(move)
        states.append(state.get_board())
        current_player = 3 - current_player

    # Define picks automáticos se não fornecidos
    if picks is None:
        picks = list(range(3, move_count + 1, 3))
        if move_count not in picks:
            picks.append(move_count)

    print(f"Capturando jogadas: {picks}")
    save_mosaic_from_states(states, picks, output_path)
    print(f"Mosaico salvo em {output_path}")


if __name__ == '__main__':
    # Executa partida com capturas automáticas
    human_vs_mcts(time_limit=1.0, output_path='mosaic.png')

#### Mosaicos Jogador vs. Árvore de Decisão

In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from game import ConnectState
from decision_tree_builder import predict
import pickle

# Carregar árvore de decisão
with open("decision_tree.pkl", "rb") as f:
    decision_tree = pickle.load(f)


def render_connect4_board(board_array, ax):
    """
    Desenha o tabuleiro de Connect4 com origem (0,0) no canto inferior-esquerdo.
    Ajusta para que row 0 seja o topo do board_array.
    """
    ax.set_facecolor('#1f77b4')
    for r in range(6):
        for c in range(7):
            cell = board_array[5 - r][c]
            color = 'white'
            if cell == 1:
                color = 'red'
            elif cell == 2:
                color = 'yellow'
            center = (c, r)
            circle = Circle(center, 0.4, edgecolor='black', facecolor=color)
            ax.add_patch(circle)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(-0.5, 6.5)
    ax.set_ylim(-0.5, 5.5)


def save_mosaic_from_states(states, picks, output_path='mosaic.png'):
    """
    Gera um mosaico de imagens a partir de uma lista de estados de jogo.
    """
    rows = cols = math.ceil(math.sqrt(len(picks)))
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 3, rows * 2.5))
    axs = axs.flatten()

    for idx, move_idx in enumerate(picks):
        board = states[move_idx - 1]
        ax = axs[idx]
        render_connect4_board(board, ax)
        ax.set_title(f'Jogada {move_idx}', pad=5)

    for ax in axs[len(picks):]:
        ax.axis('off')

    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path) or '.', exist_ok=True)
    plt.savefig(output_path)
    plt.close(fig)


def state_to_features(board_array):
    """
    Converte board_array (6x7) em dict de features para a árvore.
    """
    features = {}
    for i in range(6):
        for j in range(7):
            idx = i * 7 + j
            features[f's{idx}'] = board_array[i][j]
    return features


def human_vs_tree(picks=None, output_path='mosaic.png'):
    """
    Joga humano vs árvore de decisão, salva mosaico a cada 3 jogadas + final.

    Args:
        picks (List[int], opcional): sobrescreve capturas automáticas.
        output_path (str): arquivo de saída do mosaico.
    """
    state = ConnectState()
    states = []
    current_player = 1  # 1=humano (vermelho), 2=árvore (amarelo)
    move_count = 0

    while not state.game_over():
        move_count += 1
        state.print()

        if current_player == 1:
            # Jogada humana
            valid = False
            while not valid:
                try:
                    col = int(input("Sua vez (vermelho). Escolha coluna 0-6: "))
                    if col in state.get_legal_moves():
                        valid = True
                    else:
                        print("Coluna inválida ou cheia. Tente outra.")
                except ValueError:
                    print("Entrada inválida. Insira um número entre 0 e 6.")
            move = col
        else:
            # Jogada árvore de decisão
            print("Árvore a pensar...")
            board = state.get_board()
            features = state_to_features(board)
            move = predict(decision_tree, features)
            print(f"Árvore joga na coluna {move}.")

        state.move(move)
        states.append(state.get_board())
        current_player = 3 - current_player

    # Escolhe picks se não fornecido
    if picks is None:
        picks = list(range(3, move_count + 1, 3))
        if move_count not in picks:
            picks.append(move_count)

    print(f"Capturando jogadas: {picks}")
    save_mosaic_from_states(states, picks, output_path)
    print(f"Mosaico salvo em {output_path}")


if __name__ == '__main__':
    human_vs_tree(output_path='mosaic_tree.png')

#### Mosaicos MCTS vs. Árvore de Decisão

In [ ]:
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from game import ConnectState
from decision_tree_builder import predict
import pickle

# Carregar árvore de decisão
with open("decision_tree.pkl", "rb") as f:
    decision_tree = pickle.load(f)


def render_connect4_board(board_array, ax):
    """
    Desenha o tabuleiro de Connect4 com origem (0,0) no canto inferior-esquerdo.
    Ajusta para que row 0 seja o topo do board_array.
    """
    ax.set_facecolor('#1f77b4')
    for r in range(6):
        for c in range(7):
            cell = board_array[5 - r][c]
            color = 'white'
            if cell == 1:
                color = 'red'
            elif cell == 2:
                color = 'yellow'
            center = (c, r)
            circle = Circle(center, 0.4, edgecolor='black', facecolor=color)
            ax.add_patch(circle)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(-0.5, 6.5)
    ax.set_ylim(-0.5, 5.5)


def save_mosaic_from_states(states, picks, output_path='mosaic.png'):
    """
    Gera um mosaico de imagens a partir de uma lista de estados de jogo.
    """
    rows = cols = math.ceil(math.sqrt(len(picks)))
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 3, rows * 2.5))
    axs = axs.flatten()

    for idx, move_idx in enumerate(picks):
        board = states[move_idx - 1]
        ax = axs[idx]
        render_connect4_board(board, ax)
        ax.set_title(f'Jogada {move_idx}', pad=5)

    for ax in axs[len(picks):]:
        ax.axis('off')

    plt.tight_layout()
    os.makedirs(os.path.dirname(output_path) or '.', exist_ok=True)
    plt.savefig(output_path)
    plt.close(fig)


def state_to_features(board_array):
    """
    Converte board_array (6x7) em dict de features para a árvore.
    """
    features = {}
    for i in range(6):
        for j in range(7):
            idx = i * 7 + j
            features[f's{idx}'] = board_array[i][j]
    return features


def mcts_vs_tree(picks=None, time_limit=1.0, output_path='mosaic_mcts_tree.png'):
    """
    Joga MCTS vs árvore de decisão, salva mosaico a cada 3 jogadas + final.
    """
    state = ConnectState()
    states = []
    current_player = 2  # 1=árvore, 2=MCTS
    move_count = 0

    while not state.game_over():
        move_count += 1
        state.print()
        if current_player == 2:
            # Jogada MCTS
            print("MCTS a pensar...")
            from mcts import MCTS as Monte
            mcts = Monte(state)
            mcts.search(time_limit)
            move = mcts.best_move()
            print(f"MCTS joga na coluna {move}.")
        else:
            # Jogada árvore
            print("Árvore a pensar...")
            board = state.get_board()
            features = state_to_features(board)
            move = predict(decision_tree, features)
            print(f"Árvore joga na coluna {move}.")

        state.move(move)
        states.append(state.get_board())
        current_player = 3 - current_player

    # Picks automáticos
    if picks is None:
        picks = list(range(3, move_count + 1, 3))
        if move_count not in picks: picks.append(move_count)

    print(f"Capturando jogadas: {picks}")
    save_mosaic_from_states(states, picks, output_path)
    print(f"Mosaico salvo em {output_path}")


if __name__ == '__main__':
    mcts_vs_tree()



#### Rollouts por segundo

In [ ]:
import time
mcts = MCTS(ConnectState())
start = time.process_time()
mcts.search(time_limit=1.0)      # 1 segundo de CPU
rollouts = mcts.num_rollouts     # armazenado internamente
elapsed = time.process_time() - start
print(f"{rollouts} rollouts em {elapsed:.2f}s")

#### Tempo médio de decisão 

In [ ]:
times = []
for _ in range(10):
    t0 = time.process_time()
    mcts.search(time_limit=3.0)   # 3s solicitados
    times.append(time.process_time() - t0)
print("Média por move:", sum(times)/len(times), "s")

#### Win-Rate Contra Árvore de Decisão

In [ ]:
%load_ext autoreload
%autoreload 2
from mcts import MCTS, Node
from game import play_ai_vs_tree
N = 3
MCTS_PLAYER = 1
wins = 0
for _ in range(N):
    outcome = play_ai_vs_tree(starting_player=1, tree=decision_tree)
    if outcome == MCTS_PLAYER: wins += 1
win_rate = wins / N
print(f"Taxa de vitória do MCTS: {win_rate:.2%}")

### Gráfico de Frequência de Jogadas do MCTS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Frequência de Jogadas do MCTS

def plot_move_frequency(csv_path: str, output_path: str = None):
    """
    Lê o dataset de jogadas do MCTS e plota a frequência de cada coluna jogada.

    Args:
        csv_path: Caminho para o arquivo CSV com a coluna 'move' contendo índices de coluna (0 a 6).
        output_path: Se fornecido, salva a figura nesse caminho.
    """
    data = pd.read_csv(csv_path)
    counts = data['move'].value_counts().sort_index()
    plt.figure(figsize=(8, 5))
    counts.plot(kind='bar')
    plt.xlabel('Coluna jogada')
    plt.ylabel('Frequência')
    plt.title('Frequência de Jogadas do MCTS')
    plt.tight_layout()
    if output_path:
        plt.savefig(output_path)
    plt.show()

### Gráfico de Impacto do ALpha

In [ ]:
# 4. Impacto de α na taxa de vitória e tempo médio

def plot_alpha_impact(alphas: list, win_rates: list, avg_times: list, output_path: str = None):
    """
    Plota o impacto do parâmetro α na taxa de vitória e tempo médio por jogada.

    Args:
        alphas: Lista de valores de α testados.
        win_rates: Lista de taxas de vitória correspondentes.
        avg_times: Lista de tempos médios por jogada (s) correspondentes.
        output_path: Se fornecido, salva a figura nesse caminho.
    """
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax1.plot(alphas, win_rates, marker='o', label='Taxa de vitória')
    ax1.set_xlabel('α')
    ax1.set_ylabel('Taxa de vitória', color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()
    ax2.plot(alphas, avg_times, marker='s', linestyle='--', label='Tempo médio (s)', color='tab:orange')
    ax2.set_ylabel('Tempo médio por jogo (s)', color='tab:orange')
    ax2.tick_params(axis='y', labelcolor='tab:orange')

    plt.title('Impacto de α na taxa de vitória e tempo médio')
    fig.tight_layout()
    if output_path:
        plt.savefig(output_path)
    plt.show()

In [ ]:
# Exemplo de uso:
alphas = [0.1, 0.5, 1.0]
win_rates = [0.97, 0.99, 0.97]
avg_times = [1.90, 1.85, 2.02]
plot_alpha_impact(alphas, win_rates, avg_times, output_path='impacto_alpha.png')

### Gráfico de Trade off: Acc x Prof x Tempo

In [ ]:
# 5. Trade-off: Profundidade × Acurácia × Tempo de Previsão

def plot_depth_tradeoff(depths: list, accuracies: list, times_ms: list, output_path: str = None):
    """
    Plota o trade-off entre profundidade máxima, acurácia e tempo médio de previsão.

    Args:
        depths: Lista de profundidades máximas testadas.
        accuracies: Lista de acurácias (%) correspondentes.
        times_ms: Lista de tempos médios de previsão (ms) correspondentes.
        output_path: Se fornecido, salva a figura nesse caminho.
    """
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax1.plot(depths, accuracies, marker='o', label='Acurácia (%)')
    ax1.set_xlabel('Profundidade Máxima')
    ax1.set_ylabel('Acurácia (%)', color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')

    ax2 = ax1.twinx()
    ax2.plot(depths, times_ms, marker='x', linestyle='--', label='Tempo médio de previsão (ms)', color='tab:red')
    ax2.set_ylabel('Tempo médio de previsão (ms)', color='tab:red')
    ax2.tick_params(axis='y', labelcolor='tab:red')

    plt.title('Trade-off: Profundidade × Acurácia × Tempo de Previsão')
    fig.tight_layout()
    if output_path:
        plt.savefig(output_path)
    plt.show()


In [ ]:
depths = [2, 4, 6, 8, 10]
accuracies = [15.2, 15.7, 19.1, 20.9, 22.3]
times_ms = [3.9, 4.4, 4.2, 4.4, 6.5]
plot_depth_tradeoff(depths, accuracies, times_ms, output_path='tradeoff_profundidade.png')

### Desempenho do mcts_dataset.csv

In [ ]:
import time
import numpy as np
from decision_tree_builder import predict, id3, Node
import pandas as pd
import random
# --- 1. Carrega os dados discretizados ---
data = pd.read_csv("mcts_dataset.csv")
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

 #--- 2. Embaralha e faz o split 70/30 ---
indices = list(range(len(data)))
random.seed(42)
random.shuffle(indices)

split = int(0.7 * len(indices))
train_idx = indices[:split]
test_idx  = indices[split:]

X_train = X.iloc[train_idx].reset_index(drop=True)
y_train = y.iloc[train_idx].reset_index(drop=True)
X_test  = X.iloc[test_idx].reset_index(drop=True)
y_test  = y.iloc[test_idx].reset_index(drop=True)
tree = id3(X_train, y_train, list(X_train.columns))
# --- 3. Funções auxiliares para analisar a árvore existente ---
def count_nodes(node):
    """Conta recursivamente todos os nós (internos + folhas)."""
    return 1 + sum(count_nodes(child) for child in node.children.values())

def max_depth(node):
    """Calcula a profundidade máxima da árvore."""
    if not node.children:
        return 1
    return 1 + max(max_depth(child) for child in node.children.values())

def count_attrs_used(tree, x_dict):
    """
    Conta quantas decisões (atributos) são examinadas até chegar à folha
    para uma única instância x_dict.
    """
    node = tree
    cnt = 0
    while node.feature is not None:
        cnt += 1
        val = x_dict[node.feature]
        node = node.children.get(val, next(iter(node.children.values())))
    return cnt

# --- 3. Métricas da árvore ---
n_nodes    = count_nodes(tree)
depth_max  = max_depth(tree)

# --- 4. Precisão em treino e teste (opcional) ---
preds_train = [predict(tree, X_train.iloc[i].to_dict()) for i in range(len(X_train))]
preds_test  = [predict(tree, X_test.iloc[i].to_dict())  for i in range(len(X_test))]
train_acc   = sum(p==y for p,y in zip(preds_train, y_train))   / len(y_train)
test_acc    = sum(p==y for p,y in zip(preds_test,  y_test))    / len(y_test)

# --- 5. Tempo médio de inferência por instância ---
infer_times = []
for i in range(min(100, len(X_test))):
    x_dict = X_test.iloc[i].to_dict()
    t0 = time.process_time()
    predict(tree, x_dict)
    infer_times.append(time.process_time() - t0)
mean_infer_time = float(np.mean(infer_times))

# --- 6. Atributos usados em média ---
attrs_used = [count_attrs_used(tree, X_test.iloc[i].to_dict())
              for i in range(len(X_test))]
mean_attrs_used = float(np.mean(attrs_used))

# --- 7. Imprime resultados ---
print(f"Nº de nós da árvore:     {n_nodes}")
print(f"Profundidade máxima:     {depth_max}")
print(f"Precisão treino:         {train_acc*100:.2f} %")
print(f"Precisão teste:          {test_acc*100:.2f} %")
print(f"Tempo médio inferência:  {mean_infer_time:.4f} s")
print(f"Atributos usados (méd.):  {mean_attrs_used:.2f}")
